<a href="https://colab.research.google.com/github/CianOSull/AutoML_With_SA_FYP_2021/blob/Preprocessing/TestingNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Running Auto Keras

# Text Preprocessing Code

This area is for running the code that will process the text into an inputtable format.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Feb  4 10:54:03 2021

@author: Cian
"""
#-------------------------------------------------
#           ABOUT THIS FILE
# This file is dedicated to testing the functionality of NLTK
# 
# Useful links:
# https://www.nltk.org/
# https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
# https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92#:~:text=The%20easiest%20way%20to%20upload%20a%20CSV%20file,below%20(a%20cleaner%20method%20but%20it%E2%80%99s%20not%20necessary).
# https://medium.com/python-in-plain-english/implementing-your-first-xgboost-model-with-scikit-learn-761e2b6cfcf8
#
#-------------------------------------------------

import math
import time 
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


# To download the prequisites, run this once. 
def nltk_downloader():
  import nltk
  nltk.download('punkt')
  nltk.download('wordnet')

def clean_text(text):
    # Create the lemmatizer
    wordnet_lemmatizer = WordNetLemmatizer()
    
    # Get rid of non alpha characters except "'" as it is needed for the lemment
    text = "".join(c for c in text if c.isalnum() or c == " " or "'")
    
    # Get rid of capitals
    text = text.lower()
    
    # Tokenize the words    
    # Create tokens of each word
    token_text = word_tokenize(text)
    
    # Get rid of any piece of text that isn't over 2 characters
    token_text = [t for t in token_text if len(t) > 2] 
    
    # Put words in base form by doing lemmatization
    token_text = [wordnet_lemmatizer.lemmatize(t) for t in token_text]
    
    # Return the tokens
    return token_text


# This function will get the term frequencies for word in the review
# TF = Term I frequency in document/total words in document
def calc_tf(term_count, review_corpus):
    # A dictionary of all the term frequencies found
    tf_freq = dict.fromkeys(term_count.keys(), 0)   
    
    # Review corpus is a tokenized list so the total words iteh length
    total_words = len(review_corpus)
    
    # Calculate the term frequency for each word
    for word, count in term_count.items():
        tf_freq[word] = count/total_words
        
    return tf_freq


# This calcualtes the idf
# IDF = log(2)*(Total number of Documents/documents frequency or documents with term)
def calc_idf(unique_terms, list_doc_terms):   
    # A dicitonary of all the inverse document frequencies
    idf = dict.fromkeys(unique_terms, 0)
    
    # Basically list_doc_terms has all the documents with the term count for each word
    # You go through each document count the terms where they occured
    for doc_terms in list_doc_terms:  
        # This for loop is counting the amount of document a word was in
        for word, value in doc_terms.items():
            if 0 < value:
                idf[word] += 1
        
    # Now we calculate idf
    for word, value in idf.items():
        idf[word] = math.log10(10 / float(value))
    
    return idf

# Modified this function to return a list as dictionaries arn't needed anymore
def calc_tf_idf(tf, idf, n_terms):
    # Create an array that is of length of the number of unique terms
    tf_idf_array = np.zeros(n_terms)
    
    for index, (word, value) in enumerate(tf.items()):
        # Add the tfidf to the array
        tf_idf_array[index] = value*idf[word]
    
    return tf_idf_array


def process_text(text_data):
     # A list of all the cleaned reviews
    doc_list = []
    
    # List of all the unique terms
    unique_terms = []
    
    # A list of all the term frequencies
    tf_list = []
    
    for review in text_data:
        # First clean the review
        clean_review = clean_text(review)
        
        # Keeps track of the term counts for each word
        count_dict = {}
        
        # Now lets find the total count for each word
        for token in clean_review:
            if token not in count_dict:
                count_dict[token] = 1
            else:
                count_dict[token] += 1
        
        # Caclulate the term frequencies for each document
        tf_list.append(calc_tf(count_dict, clean_review))
        
        # Then add the dictionary of counts for each document to the list
        doc_list.append(count_dict)
        
        # Then add the new unique terms
        unique_terms = set(unique_terms).union(set(clean_review))
    
    # Calculate the inverse document frequency value
    idf = calc_idf(unique_terms, doc_list)
    
    # This array will contain the tfidf values for each term in each review
    tfidf_values = np.zeros((len(tf_list), len(unique_terms)))
    
    # Now we can get the TFIDF for each document
    for index, term_freq in enumerate(tf_list):
        # This will return an array of the tfidf values calculated.
        # The length of the unique terms list is passed in so that the 
        # Array that is returned matches the tfidf array
        tf_idf_array = calc_tf_idf(term_freq, idf, len(unique_terms))
        # Add this to the overall tfidf values calculated
        tfidf_values[index,:] = tf_idf_array
    
    return tfidf_values
    

# Testing nltk on the dataset
def dataset_testing():
    print("="*50)

    # Load the dataset
    # imdb_df = pd.read_csv("IMDB Dataset.csv")
    imdb_df = pd.read_csv("/content/drive/MyDrive/CIT/FYP/ImplementationFiles/IMDB_Dataset.csv")
    print("Dataset loaded")
    print("="*50)

    
    # Change each positive and negative value to 1 and 0 respectively    
    imdb_df['sentiment'] = imdb_df['sentiment'].map({'negative' : 0, 'positive' : 1})
    
    # For testing, a much smaller dataset is going to be used
    imdb_df = imdb_df.head(5000)

    # Group all the negative reviews and get the first 2500
    imdb_df_neg = (imdb_df[imdb_df['sentiment'] == 0])[0:2500]
    # Group all the positive and get the first 2500
    imdb_df_pos = imdb_df[imdb_df['sentiment'] == 1]
    
    test_df = pd.concat([imdb_df_neg, imdb_df_pos]) 
    # print(test_df)
    
    # .values on a column of a dataframe returns a numpy array
    # This is a numpy array of all the reviews
    # initial_reviews = imdb_df['review'].values
    initial_reviews = test_df['review'].values
    
    # This is a numpy array of all the positive and negativelabels
    # labels = imdb_df['sentiment'].values
    labels = test_df['sentiment'].values
    
    print("Creating Feature Vector")
    print("="*50)
    start = time.time()
    # Process the text data and create teh feature vector
    feature_vector = process_text(initial_reviews)
    end = time.time()
    print("Feature Vector Created")
    print(f"Execution time is {end - start} secs")
    print("="*50)
    
    # Shuffle the labesl and feature vector using sklearn shuffle
    feature_vector, labels = shuffle(feature_vector, labels)
    
    # Creating train and test data
    # Inital split will be 80:20 just for testing
    no_samples = 0.8
    
    # This gets the percentage of indexes from feature vector and uses those for training
    X_train = feature_vector[0:int(no_samples*len(feature_vector))]
    y_train = labels[0:int(no_samples*len(labels))]
    
    # Go from the index that was used for training to the final
    X_test = feature_vector[int(no_samples*len(feature_vector)):len(feature_vector)]
    y_test = labels[int(no_samples*len(labels)):len(labels)]
    
    # Run a Logistic regression model just for a quick test
    # model = LogisticRegression()
    
    # XGB can be a better generic tester so it is being used instead.
    model = XGBClassifier()

    print("Creating Model")
    print("="*50)
    start = time.time()
    try:
      model.fit(X_train, y_train)
    except Exception as e:
      print(e)
    end = time.time()
    print("Model Created")
    print(f"Execution time is {end - start} secs")
    print("="*50)
    
    # print("Train accuracy:", model.score(X_train, y_train))
    # print("Test accuracy:", model.score(X_test, y_test))

    # Test the model
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]

    # evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    print("="*50)

    print("Model Parameters:\n", model)

def main():
    # Download prequisites
    nltk_downloader()
    
    # Test information on dataset
    dataset_testing()
    

# Run Main
main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Dataset loaded
Creating Feature Vector
Feature Vector Created
Execution time is 32.44350337982178 secs
Creating Model
Model Created
Execution time is 354.90329456329346 secs
Accuracy: 52.21%
Model Parameters:
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


# Google Collab Test Area

In [ ]:
print("Hello World")

In [ ]:
from sklearn import datasets
from sklearn import metrics
from sklearn import linear_model
from sklearn import model_selection

def example_mnist():
  digits = datasets.load_digits()
  targets = set(digits.target)

  train_data = digits.data[0:int(0.8*len(digits.data))]
  train_target = digits.target[0:int(0.8*len(digits.target))]
  test_data = digits.data[int(0.8*len(digits.data)):len(digits.data)]
  test_target = digits.target[int(0.8*len(digits.target)):len(digits.target)]

  kf = model_selection.KFold(n_splits=2, shuffle=True)

  best_score = 1e-10

  for train_index,test_index in kf.split(train_data):
      clf = linear_model.Perceptron()

      clf.fit(train_data[train_index], train_target[train_index ])
      prediction1 = clf.predict(train_data[test_index])

      score = metrics.accuracy_score(train_target[test_index], prediction1)
      print("Perceptron accuracy score: ", score)
      print()

      if score > best_score:
          best_clf = clf


  prediction = best_clf.predict(test_data)
  for digit in targets:
      print(digit, " -> ", sum(test_target[prediction!=test_target]==digit))
  
  
def main():
  print("="*50)
  example_mnist()

main()